<a href="https://colab.research.google.com/github/guilherme-argentino/fiap-ia4devs-techchallenge-fase3/blob/main/Fase3_TechChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tech Challenge Fase 3

Vamos usar o modelo **BERT** no lugar de GPT-2, especificamente o modelo **`bert-base-uncased`** da Hugging Face, neste processo de fine-tuning muda um pouco, já que o BERT é um modelo "Masked Language Model" (MLM), e geralmente é utilizado para tarefas como classificação, predição de token, ou tarefas como QA (perguntas e respostas).

Vamos montar o notebook para usar o **BERT**. O objetivo é ajustar o modelo para responder a perguntas com base nas descrições dos produtos, o que pode ser tratado como uma tarefa de classificação de sequência (entrada e saída).


### 1. Instalar dependências

Primeira célula: Instala as bibliotecas necessárias.

In [1]:
!pip install datasets transformers

### 2. Carregar e ler o arquivo JSONL com pandas

Usando o pandas para carregar o arquivo `trn.json` no formato JSONL. Certifique-se de fazer upload do arquivo no Colab, ou montar o drive e fornecer o caminho correto.


In [2]:
from datasets import load_dataset

# Carregar o arquivo trn.json
jsonl_file = '/content/drive/MyDrive/FIAP/1IADT/Fase-3/LF-Amazon-1.3M/trn.json'  # Ajuste o caminho conforme necessário

# Ler o arquivo jsonl com pandas
dataset = load_dataset('json', data_files=jsonl_file, split='train', streaming=True)

# Exibir uma amostra dos dados
print(next(iter(dataset)))

{'uid': '0000031909', 'title': 'Girls Ballet Tutu Neon Pink', 'content': 'High quality 3 layer ballet tutu. 12 inches in length', 'target_ind': [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 111], 'target_rel': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}


### 3. Processar o Dataset e Criar Prompts
Agora, vamos processar o dataset e criar os prompts baseados no título e descrição do produto. O input será o título do produto, e o output será a descrição.

In [3]:
# Inicializar uma lista para armazenar os dados processados
dados_processados = []

# Iterar sobre o dataset e criar os pares de entrada e saída
for example in dataset:
    title = example.get('title', '')
    content = example.get('content', '')

    # Criar um prompt com base no título e na descrição
    prompt = f"Descreva o produto com o título '{title}'?"
    dados_processados.append({"input_text": prompt, "output_text": content})

    # Opcionalmente, pare após processar um certo número de exemplos
    if len(dados_processados) >= 10000:
        break  # Ajuste este valor conforme necessário para processar mais exemplos

# Mostrar uma amostra dos dados processados
dados_processados[:2]


[{'input_text': "Descreva o produto com o título 'Girls Ballet Tutu Neon Pink'?",
  'output_text': 'High quality 3 layer ballet tutu. 12 inches in length'},
 {'input_text': "Descreva o produto com o título 'Adult Ballet Tutu Yellow'?",
  'output_text': ''}]

### 4. Tokenizar os Dados para o BERT
Agora, vamos tokenizar os pares `input_text` e `output_text` usando o tokenizer do **BERT**.


In [9]:
from transformers import AutoTokenizer

# Carregar o tokenizer do BERT (bert-base-uncased)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Função para tokenizar o texto de entrada e saída
def tokenize_function(examples):
    inputs = tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(examples['output_text'], padding="max_length", truncation=True, max_length=512)
    inputs['labels'] = outputs['input_ids']  # Usar os input_ids como rótulos
    return inputs

# Aplicar o tokenizer aos dados processados
dados_tokenizados = [tokenize_function(example) for example in dados_processados]

# Exemplo de um registro tokenizado
dados_tokenizados[0]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'input_ids': [101, 4078, 16748, 3567, 1051, 4013, 8566, 3406, 4012, 1051, 14841, 8525, 4135, 1005, 3057, 7250, 10722, 8525, 16231, 5061, 1005, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### 5. Converter para o Dataset da Hugging Face
Agora que temos os dados tokenizados, vamos convertê-los em um objeto `Dataset` da Hugging Face para prepará-los para o treinamento.

In [10]:
from datasets import Dataset

# Converter os dados tokenizados em um Dataset da Hugging Face
hf_dataset = Dataset.from_list(dados_tokenizados)

# Mostrar um exemplo do dataset
hf_dataset[0]

{'input_ids': [101,
  4078,
  16748,
  3567,
  1051,
  4013,
  8566,
  3406,
  4012,
  1051,
  14841,
  8525,
  4135,
  1005,
  3057,
  7250,
  10722,
  8525,
  16231,
  5061,
  1005,
  1029,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

### 6. Fine-Tuning do BERT
Vamos agora usar o modelo **BERT** (`bert-base-uncased`) e a API `Trainer` da Hugging Face para fazer o fine-tuning do modelo usando o dataset tokenizado.

In [11]:
from transformers import BertForMaskedLM, Trainer, TrainingArguments

# Carregar o modelo BERT para Masked Language Modeling
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

# Definir os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=500,
    save_steps=1000,
    num_train_epochs=3,
    save_total_limit=2,
)

# Inicializar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_dataset,
)

# Treinar o modelo
trainer.train()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss


ValueError: Trainer: evaluation requires an eval_dataset.

### 7. Configurar o treinamento usando o Trainer da Hugging Face

Agora configuramos o treinamento. Ajustamos os argumentos de treinamento, como o número de épocas e o tamanho do batch.

In [ ]:
from transformers import Trainer, TrainingArguments

# Definir os parâmetros de treinamento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
)

# Configurar o Trainer para o BERT
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Executar o treinamento
trainer.train()

### 8. Avaliação do modelo após o fine-tuning

Após o treinamento, podemos testar o modelo para gerar uma descrição com base em um novo título fornecido pelo usuário.

In [ ]:
# Pergunta do usuário
user_input = "Descreva o produto com o título 'Headphones Bluetooth XYZ'?"

# Tokenizar a pergunta
inputs = tokenizer(user_input, return_tensors="pt")

# Gerar a resposta com o modelo fine-tunado
outputs = model(**inputs)

# Exibir o resultado
print("Saída:", outputs)

### 9. Conclusão e Salvamento do Modelo

Por fim, você pode salvar o modelo treinado para usá-lo posteriormente.

In [ ]:
# Salvar o modelo fine-tunado
model.save_pretrained("./fine_tuned_bert_model")
tokenizer.save_pretrained("./fine_tuned_bert_model")

print("Modelo treinado salvo!")